# Modeling 



## Table of Contents
1. [Introduction](#Introduction)
2. [Data Preparation](#Data-Preparation)
3. [Modeling process](#Modeling-Process)
4. [Key Metrics](#Key-Metrics)
5. [Results](#Results)



# Introduction <a id="Introduction"></a>
The goal of the project is to provide Swire Coca-Cola with the ability to predict future unplanned maintenance events and minimize downtime. Unplanned maintenance can lead to significant operational losses. Our primary analytical challenge will be identifying the variables that can predict machine breakdowns. Key target variables will include actual work in minutes and maintenance activity type.

# Data Preparation <a id="Data-Preparation"></a>

In [1]:
# Package Importing and Data Cleaning 
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
df = pd.read_csv('IWC_Work_Orders_Extract.csv', low_memory=False)

#remove columns not needed 

columns_to_remove = [
    'EQUIPMENT_DESC', 
    'EQUIP_CAT_DESC', 
    'EQUIP_START_UP_DATE', 
    'EQUIP_VALID_FROM', 
    'EQUIP_VALID_TO', 
    'EXECUTION_FINISH_DATE', 
    'ACTUAL_START_TIME', 
    'ACTUAL_FINISH_TIME', 
    'EXECUTION_START_DATE'
]

df = df.drop(columns=columns_to_remove)

# Convert date columns to datetime
df['ACTUAL_WORK_IN_MINUTES'] = pd.to_numeric(df['ACTUAL_WORK_IN_MINUTES'], errors='coerce')

# Remove duplicate rows
df.drop_duplicates(inplace=True)


#take a quick look at the data to ensure it looks good 
print("Columns:", df.columns.tolist(), "\n", df.head())



Columns: ['ORDER_ID', 'PLANT_ID', 'PRODUCTION_LOCATION', 'ACTUAL_WORK_IN_MINUTES', 'MAINTENANCE_PLAN', 'MAINTENANCE_ITEM', 'MAINTENANCE_ACTIVITY_TYPE', 'ORDER_DESCRIPTION', 'MAINTENANCE_TYPE_DESCRIPTION', 'FUNCTIONAL_LOC', 'FUNCTIONAL_AREA_NODE_1_MODIFIED', 'FUNCTIONAL_AREA_NODE_2_MODIFIED', 'FUNCTIONAL_AREA_NODE_3_MODIFIED', 'FUNCTIONAL_AREA_NODE_4_MODIFIED', 'FUNCTIONAL_AREA_NODE_5_MODIFIED', 'EQUIPMENT_ID'] 
     ORDER_ID PLANT_ID PRODUCTION_LOCATION  ACTUAL_WORK_IN_MINUTES  \
0  705642457     G812                ROMA                   390.0   
1  704191697     G812                ROMA                   420.0   
2  704466547     G812                ROMA                    30.0   
3  703834477     G812                ROMA                    60.0   
4  704661125     G291               MONZA                   120.0   

  MAINTENANCE_PLAN  MAINTENANCE_ITEM MAINTENANCE_ACTIVITY_TYPE  \
0              NaN               NaN                 Unplanned   
1              NaN               NaN 

# Modeling Process A <a id="Modeling-Process"></a>
Let’s examine the relationship between planned and unplanned work, as well as key problem areas, so that we can build a useful tool based on real-world pain points.

In [ ]:
# Calculate the ratio of "unplanned" to "planned" maintenance activities
maintenance_activity_counts = df['MAINTENANCE_ACTIVITY_TYPE'].value_counts()
unplanned_count = maintenance_activity_counts['Unplanned'] if 'Unplanned' in maintenance_activity_counts else 0
planned_count = maintenance_activity_counts['Planned'] if 'Planned' in maintenance_activity_counts else 0
ratio = unplanned_count / (planned_count + unplanned_count) if planned_count + unplanned_count > 0 else 0

print(f"Ratio of 'unplanned' to 'planned' maintenance activities: {ratio}")



Ratio of 'unplanned' to 'planned' maintenance activities: 0.8946326678175867


### Insights
For every one planned maintenance activity, there are approximately 0.8946 unplanned maintenance activities. Let's examine the counts as well to get a better understanding of the scale.

In [3]:
# Count the number of occurrences of each value in the MAINTENANCE_ACTIVITY_TYPE column
counts = df['MAINTENANCE_ACTIVITY_TYPE'].value_counts()

# Print the counts
print(counts)



Unplanned    1276877
Planned       150387
Name: MAINTENANCE_ACTIVITY_TYPE, dtype: int64


Let’s examine which plants have the highest ratio of planned to unplanned maintenance activities.

In [4]:

# Group the data by PLANT_ID and calculate the counts of 'unplanned' and 'planned' maintenance activities
grouped_df_pl = df.groupby('PRODUCTION_LOCATION')['MAINTENANCE_ACTIVITY_TYPE'].value_counts().unstack()

# Rename the columns
grouped_df_pl.columns = ['Planned', 'Unplanned']

# Calculate the ratio of 'unplanned' to 'planned' maintenance activities
grouped_df_pl['Ratio'] = grouped_df_pl['Unplanned'] / grouped_df_pl['Planned']

# Print the dataframe
print(grouped_df_pl)





                     Planned  Unplanned      Ratio
PRODUCTION_LOCATION                               
COTA                   26338      89965   3.415787
MONACO                  8150      76544   9.391902
MONZA                  48658     176621   3.629845
ROMA                   21222      57888   2.727735
SILVERSTONE            27809     603094  21.687008
SUZUKA                 18210     272765  14.978858


Silverstone and Suzuka are both outliers, showing underperformance. Now that we have a broad understanding, let’s identify the specific functional locations that would benefit the most from implementing more planned maintenance.

In [5]:


# Group the data by PLANT_ID and calculate the counts of 'unplanned' and 'planned' maintenance activities
grouped_df_fM = df.groupby('FUNCTIONAL_LOC')['MAINTENANCE_ACTIVITY_TYPE'].value_counts().unstack()

# Rename the columns
grouped_df_fM.columns = ['Planned', 'Unplanned']

# Calculate the ratio of 'unplanned' to 'planned' maintenance activities
grouped_df_fM['Ratio'] = grouped_df_fM['Unplanned'] / grouped_df_fM['Planned']

# Sort the dataframe by the Ratio column in descending order
grouped_df_fMf = grouped_df_fM.sort_values(by='Ratio', ascending=False)

# Print the dataframe
print(grouped_df_fMf)





                       Planned  Unplanned       Ratio
FUNCTIONAL_LOC                                       
G221-PRD-L30-L06-L010      1.0      209.0  209.000000
G221-PRD-L30-L02-L070      3.0      511.0  170.333333
G221-PRD-L15-L04-L200      1.0      145.0  145.000000
G811-PRD-A55-I04           1.0      136.0  136.000000
G221-PRD-L30-L06-L070      1.0      131.0  131.000000
...                        ...        ...         ...
G816-PRD-P90-W02-S090      NaN        1.0         NaN
G816-PRD-P90-W06           NaN        2.0         NaN
G816-PRD-P90-W10           NaN        5.0         NaN
G816-PRD-P90-XXX-S210      NaN        5.0         NaN
G999-SLC-PRD               NaN        1.0         NaN

[667 rows x 3 columns]


G221-PRD-L30-L02-L070 has a poor ratio and a slightly larger sample size, so let's use that one.


# Key Metrics <a id="Key-Metrics"></a>
For the functional location G221-PRD-L30-L02-L070, the ratio of planned to unplanned maintenance events is 3:511, or approximately 170.

Let’s create a calculator to determine the optimal amount of planned maintenance needed to minimize costs for our selected problematic location. Since we do not have access to financial data on a per-hour basis, we will use the following assumptions: 50 Dollors per hour for planned maintenance and 500 Dollord per hour for unplanned maintenance, which includes losses from not stopping production. Additionally, we need to enter the ratio of planned to unplanned work, which in our case is 170.

In [ ]:
def break_even_point_calculator():
    # Get the user input
    cost_per_hour_planned = float(input("Enter the monetary cost per hour of planned maintenance: $"))
    cost_per_hour_unplanned = float(input("Enter the monetary cost per hour of unplanned maintenance (including compensation for lost production): $"))
    ratio_planned_to_unplanned = float(input("Enter the ratio of planned to unplanned maintenance events (e.g. 170.33 for 3 planned and 511 unplanned events): "))

    # Calculate the break-even point
    break_even_point = (cost_per_hour_unplanned / cost_per_hour_planned) * (1 / ratio_planned_to_unplanned)

    print(f"The break-even point is: {break_even_point:.2f} hours of planned maintenance per hour of unplanned maintenance.")

    # Calculate the number of hours of planned maintenance needed
    hours_planned = break_even_point * ratio_planned_to_unplanned
    print(f"The number of hours of planned maintenance needed is: {hours_planned:.2f} hours.")

    # Calculate the cost of planned maintenance
    cost_planned = hours_planned * cost_per_hour_planned
    print(f"The cost of planned maintenance is: ${cost_planned:.2f}.")

    # Calculate the value of unplanned maintenance avoided
    value_avoided = cost_per_hour_unplanned * ratio_planned_to_unplanned
    print(f"The value of unplanned maintenance avoided is: ${value_avoided:.2f}.")

break_even_point_calculator()

# Results <a id="Results"></a>
According to the break-even analysis, Swire should perform 10 hours of planned maintenance for the functional location G221-PRD-L30-L02-L070, which will cost them 500 Dollors. By doing so, they will save 85,000 Dollors. While these savings may seem quite extreme, we selected one of the least efficient functional locations and made certain assumptions regarding the cost per hour. When applied to functional locations with more average performance, the results will normalize.